In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('https://storage.googleapis.com/gmtempor/reco_sample_dataset.csv')

In [18]:
scenario1 = df.drop(['partition_date', 'orderparentid','productcontentid','category_id','price','color_id','business_unit'], axis=1)

In [20]:
import sqlite3

conn = sqlite3.connect(":memory:")

In [22]:
scenario1.to_sql("scenario1",  conn, if_exists="replace")

In [50]:
user = pd.read_sql("""
select user_id, category_name,count(1) n from scenario1 group by user_id,category_name order by n desc;
""", conn)

In [51]:
category = pd.read_sql(""" 
select category_name, brand_id, count(1) n from scenario1 group by category_name, brand_id order by n desc;
""", conn)

In [52]:
category['weight'] = category.groupby('category_name').transform(lambda x: x/x.sum())[['n']]

In [53]:
category.head()

,category_name,brand_id,n,weight
0,T-Shirt,40,15947,0.347960
1,Elbise,40,11088,0.534980
2,T-Shirt,15508,6899,0.150535
3,Bluz,40,5354,0.477737
4,T-Shirt,7651,5062,0.110452


In [54]:
user['weight'] = user.groupby('user_id').transform(lambda x: x/x.sum())[['n']]

In [55]:
user

,user_id,category_name,n,weight
0,85115,T-Shirt,670,0.104280
1,85115,Gömlek,503,0.078288
2,85115,Sweatshirt,304,0.047315
3,85115,Sneaker,289,0.044981
4,85115,Kazak,265,0.041245
...,...,...,...,...
363003,112536,Pratik Temizlik Gereci,1,0.166667
363004,112538,Amino Asitler,1,0.166667
363005,112538,Hırdavat Ürünleri,1,0.166667
363006,112538,Vitaminler,1,0.166667


In [56]:
ranking = user.merge(category, on="category_name")

In [57]:
ranking

,user_id,category_name,n_x,weight_x,brand_id,n_y,weight_y
0,85115,T-Shirt,670,0.104280,40,15947,0.347960
1,85115,T-Shirt,670,0.104280,15508,6899,0.150535
2,85115,T-Shirt,670,0.104280,7651,5062,0.110452
3,85115,T-Shirt,670,0.104280,842,2459,0.053655
4,85115,T-Shirt,670,0.104280,3268,2306,0.050316
...,...,...,...,...,...,...,...
49024017,111769,Masa Tenisi Topu,1,0.200000,15451,1,1.000000
49024018,111942,Bahçe Dolabı,1,0.012500,34975,1,0.500000
49024019,111942,Bahçe Dolabı,1,0.012500,974457,1,0.500000
49024020,112221,Bahçe Dolabı,1,0.028571,34975,1,0.500000


In [58]:
ranking['weight'] = ranking.weight_x * ranking.weight_y

In [61]:
alpha = 0.9

ranking['weight'] = alpha * ranking.weight_x + (1-alpha) * ranking.weight_y

In [63]:
user_id = 85115


all_user_recommendations = ranking[['user_id','category_name','brand_id', 'weight']][ranking.user_id == user_id]

In [64]:
all_user_recommendations.head(3)

,user_id,category_name,brand_id,weight
0,85115,T-Shirt,40,0.128648
1,85115,T-Shirt,15508,0.108906
2,85115,T-Shirt,7651,0.104897


In [73]:
recos = df.merge(all_user_recommendations, on=['category_name','brand_id']).sort_values('weight',ascending=False)

In [76]:
recos.head()

,partition_date,orderparentid,user_id_x,productcontentid,brand_id,category_id,category_name,gender,price,color_id,business_unit,user_id_y,weight
11423,2020-08-06 20:00:00,328127177,111083,4780247,40,604,T-Shirt,Kadın,39.99,14.0,PL Woman,85115,0.128648
10711,2020-08-17 23:00:00,333145181,30720,5906042,40,604,T-Shirt,Kadın,34.99,4.0,PL Woman,85115,0.128648
10698,2020-08-05 23:00:00,327723005,44583,35467152,40,604,T-Shirt,Kadın,39.99,8.0,PL Woman,85115,0.128648
10699,2020-08-05 23:00:00,327723005,44583,34696137,40,604,T-Shirt,Kadın,29.99,3.0,PL Woman,85115,0.128648
10700,2020-08-06 10:00:00,327862045,87208,34696137,40,604,T-Shirt,Kadın,29.99,3.0,PL Woman,85115,0.128648
